In [5]:
import torch
import torch.nn as nn

In [8]:
D = 2
H = 5
O = 2

In [6]:
device = 'cuda' if torch.cuda.is_available() else 'cpu'

In [12]:
from lib.quantumsearch import QuantumSearch


def create_model(max_depth, beam_width, branching_width):
    encoder = nn.Sequential(
        nn.Linear(D, H),
        nn.ReLU(),
        nn.LayerNorm(H)
    )

    search = QuantumSearch(
        transition=nn.Sequential(
            nn.Linear(H, H),
            nn.ReLU(),
            nn.Linear(H, 2*H),
        ),
        fitness=nn.Sequential(
            nn.Linear(H, H),
            nn.ReLU(),
            nn.Linear(H, 1),
        ),
        # # sample=nn.Sequential(
        # #     SampleUniform(H, num_samples=num_samples),
        # #     nn.LayerNorm(H)
        # ),
        max_depth=max_depth,
        branching_width=branching_width,
        beam_width=beam_width
    )

    decoder = nn.Sequential(
        nn.Linear(H, O)
    )

    model = nn.Sequential(
        encoder,
        search,
        decoder
    )

    return model

model = create_model(1, 8, 8)
model.to(device)

Sequential(
  (0): Sequential(
    (0): Linear(in_features=2, out_features=5, bias=True)
    (1): ReLU()
    (2): LayerNorm((5,), eps=1e-05, elementwise_affine=True)
  )
  (1): QuantumSearch(
    (transition): Sequential(
      (0): Linear(in_features=5, out_features=5, bias=True)
      (1): ReLU()
      (2): Linear(in_features=5, out_features=10, bias=True)
    )
    (fitness): Sequential(
      (0): Linear(in_features=5, out_features=5, bias=True)
      (1): ReLU()
      (2): Linear(in_features=5, out_features=1, bias=True)
    )
  )
  (2): Sequential(
    (0): Linear(in_features=5, out_features=2, bias=True)
  )
)

In [13]:
from lib.quantumsearch import FitnessFunction, OneToManyNetwork, QuantumSearch
from lib.quantumsearch import TransitionFunction

D = 2
H = 5
O = 3

encoder = nn.Sequential(
    nn.Linear(D, H),
    nn.ReLU(),
    nn.LayerNorm(H)
)

class Unpack(nn.Module):
    def __init__(self, size):
        super(Unpack, self).__init__()
        self.size = size

    def forward(self, x):
        return x.reshape(*x.shape[:-1], x.shape[-1]//self.size, self.size)

search = QuantumSearch(
    transition=TransitionFunction(
        OneToManyNetwork(
            nn.Sequential(
                nn.Linear(H, H),
                nn.ReLU(),
                nn.Linear(H, 3*H),
                Unpack(3)
            )
        ),
    ),
    fitness=FitnessFunction(
        OneToManyNetwork(
            nn.Sequential(
                nn.Linear(H, H),
                nn.ReLU(),
                nn.Linear(H, 3),
                Unpack(3)
            )
        ),
    ),
    max_depth=3,
    beam_width=3, # redundant since we have deterministic one-to-many functions
    branching_width=3 # redundant since we have deterministic one-to-many functions
)

decoder = nn.Sequential(
    nn.Linear(H, 3)
)

model = nn.Sequential(
    encoder,
    search,
    decoder
)

ImportError: cannot import name 'FitnessFunction' from 'lib.quantumsearch' (/home/banashree/neural-search/lib/quantumsearch.py)